# Tutorial 8 - Extending built in classes

This tutorial covers how to extend built-in classes to further customize your use of the package to suit your own unique data processing/anaysis needs.

Let's say there is a particular analysis use-case that is not provided as a built-in in the current package release, by following the object-oriented structure of the package you can quickly create and add methods to extend the functionality of all existing class structures.

#### Creating a custom class for Interneuron Gcamp imaging during seizures

In the following example, we create a custom `Experiment` class by using the inheritance principles of Python. We create a new `Experiment` class called `InterneuronExp` that extends the built-in `Experiment` class.


In this experiment, Gcamp imaging was performed in Nkx2.1-cre-mCherry mice which label a sub-type of interneurons with mCherry. There were also two experimental phases of imaging: pre-4ap and post-4ap. We require a number of functionalities in this class that are not available in the built-in `Experiment` class structure:
1) a list of trials in the pre4ap injection phase
2) a list of trials in the post4ap injection phase
3) a list of Suite2p ROIs that are also interneurons
3a) add this list as a `obs` entry into the `anndata` storage of all trials

We also need to further create a custom `Trial` class to store trial-level data that is not allowed using built-in methods/attributes. This is demonstrated further below.

In [ ]:
import packerlabimaging as pli


class InterneuronExp(pli.Experiment):
    def __init__(self, initalization_dict):

        super().__init__(**initalization_dict)  # call to initialize the super-class (i.e. `pli.Experiment`)

        # set pre4ap and post4ap experiments:
        self.pre4ap_trials = []
        for trial in self.trialIDs:
            self.pre4ap_trials.append(trial) if 'pre 4ap' in self.TrialsInformation[trial]['expGroup'] else None

        self.post4ap_trials = []
        for trial in self.trialIDs:
            self.post4ap_trials.append(trial) if 'post 4ap' in self.TrialsInformation[trial]['expGroup'] else None

In [ ]:
# create the initialization dictionary containing information about trials within this experiment

prep = 'PS12'
date = '2021-01-25'

ExperimentMetainfo = {
    'dataPath': f'/home/pshah/mnt/qnap/Data/{date}',  # todo this seems very vauge, maybe add very specific documentation about what this is supposed to be, or just say tiff path?
    'analysisSavePath': f'/home/pshah/mnt/qnap/Analysis/{date}/{prep}/',
    "expID": prep,
    'date': date,
    'comments': f'{prep} - interneuron gcamp imaging + LFP pre- and post-4ap'
}

# create Experiment using custom class
expobj = InterneuronExp(initalization_dict=ExperimentMetainfo)




In [ ]:
# create initialization dictionary to initialize each trial and add trials to the previously created Experiment
trials_list_pre4ap = ['t-001', 't-002', 't-003']
# todo - add functionality to add longer detailed comments for each trial (e.g. t-001: 30 mins spont, t-002: 30 mins spont + LFP, etc.) (other than expGroup)

for idx, trial in enumerate(trials_list_pre4ap):
    data_path_base = f'/home/pshah/mnt/qnap/Data/{date}'
    TwoPhotonImagingMetainfo = {'date': date,
                                'trial_id': trial,
                                'exp_id': prep,
                                'microscope': 'Bruker 2pPlus',
                                'tiff_path': f'{data_path_base}/{date}_{trial}/{date}_{trial}_Cycle00001_Ch3.tif',
                                'save_dir': expobj.analysisSavePath,
                                'expGroup': "pre 4ap 2p imaging",
                                'PaqInfoTrial': {'paq_path': f'{data_path_base}/{date}_{prep}_{trial[2:]}.paq',  # path to the .paq files for the selected trials
                                                 'frame_channel': 'frame_clock'}
                                }

    trialobj = pli.TwoPhotonImagingTrial(**TwoPhotonImagingMetainfo)

    # add each Trial to the overall Experiment using the trialobj
    expobj.add_trial(trialobj=trialobj)


trials_list_post4ap = ['t-006', 't-007', 't-008', 't-009']
for idx, trial in enumerate(trials_list_post4ap):
    data_path_base = f'/home/pshah/mnt/qnap/Data/{date}'
    TwoPhotonImagingMetainfo = {'date': date,
                                'trial_id': trial,
                                'exp_id': prep,
                                'microscope': 'Bruker 2pPlus',
                                'tiff_path': f'{data_path_base}/{date}_{trial}/{date}_{trial}_Cycle00001_Ch3.tif',
                                'save_dir': expobj.analysisSavePath,
                                'expGroup': "post 4ap 2p imaging",
                                'PaqInfoTrial': {'paq_path': f'{data_path_base}/{date}_{prep}_{trial[2:]}.paq',  # path to the .paq files for the selected trials
                                                 'frame_channel': 'frame_clock'}
                                }

    trialobj = pli.TwoPhotonImagingTrial(**TwoPhotonImagingMetainfo)

    # add each Trial to the overall Experiment using the trialobj
    expobj.add_trial(trialobj=trialobj)

